In [2]:
import numpy as np
from torchvision.datasets import MNIST

/home/beckmann/miniconda3/envs/deepfake/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
train_data = MNIST(root="./", train=True, transform=None, target_transform=None, download=True)
print(len(train_data))

60000


In [3]:
img = train_data.__getitem__(0)[0]
display(img)

In [4]:
for i in range(1,10):
    img = train_data.__getitem__(i)[0]
    display(img)

In [5]:
img = np.array(train_data.__getitem__(0)[0])
label = np.array(train_data.__getitem__(0)[1])

In [10]:
class conv_layer():
    def __init__(self, kernel_size: int, zero_padding_size: int, stride: int, weights: list = None):
        self.kernel_size = kernel_size
        self.zero_padding_size = zero_padding_size
        self.stride = stride

        if weights is not None:
            self.weights = np.array(weights)
            self.kernel_size = len(weights)
        else:
            self.weights = np.random.rand(kernel_size, kernel_size)

    def forward(self, input):
        batch_size = input.shape[0]
        input_shape = input.shape[1]
        output_shape = int(np.ceil((input_shape + 2*self.zero_padding_size - self.kernel_size) / self.stride) + 1)

        padded_input = np.pad(input, ((0, 0), (self.zero_padding_size, self.zero_padding_size), (self.zero_padding_size, self.zero_padding_size)))
        output = np.zeros((batch_size, output_shape, output_shape))

        for b in range(batch_size):
            for i in range(output_shape):
                for j in range(output_shape):
                    patch = padded_input[b, i*self.stride:i*self.stride+self.kernel_size, j*self.stride:j*self.stride+self.kernel_size]
                    output[b, i, j] = np.sum(patch*self.weights)

        return output

In [ ]:
input = np.array([[1,1,1,0,0], [0,1,1,1,0], [0,0,1,1,1], [0,0,1,1,0], [0,1,1,0,0]])
input = np.expand_dims(input, 0)
weights = [[0,0,1], [0,1,1], [1,1,1]]

layer = conv_layer(3, 1, 1, weights)
output = layer.forward(input)

print(output)


In [5]:
class fc_layer():
    def __init__(self, in_channels: int, out_channels: int, weights: list = None):
        self.in_channels = in_channels
        self.out_channels = out_channels
        
        if weights is not None:
            self.weights = np.array(weights)
        else:
            self.weights = np.random.rand(self.in_channels, self.out_channels)

    def forward(self, input):
        return np.matmul(input, self.weights)

In [6]:
def relu(x):
    return x * (x > 0)

def softmax(x):
    exp_vals = np.exp(x - np.max(x, axis=-1, keepdims=True))
    return exp_vals / np.sum(exp_vals, axis=-1, keepdims=True)

In [23]:
class simple_cnn():
    def __init__(self, conv_layer, fc_layer):
        self.conv_layer = conv_layer
        self.fc_layer = fc_layer

    def forward(self, input):
        features = self.conv_layer.forward(input)
        features = relu(features)
        features = features.reshape(features.shape[0], features.shape[1]**2)
        output = self.fc_layer.forward(features)
        output = softmax(output)

        return output

In [24]:
input = np.random.rand(2, 28, 28)
cl = conv_layer(3, 1, 1, None)
fc = fc_layer(28**2, 10)
cnn = simple_cnn(cl, fc)
out = cnn.forward(input)



2 28 28
(2, 10)


In [16]:
def label_one_hot_encoding(x, dim=10):
    output = np.zeros(dim)
    output[x] = 1
    return output

In [17]:
def mse(target, pred):
    return np.mean(np.sum((target-pred)**2))

In [21]:
def backward_mse_to_fc(input, target, pred, fc_weights):
    gradients = np.zeros(fc_weights.shape)
    batch_size = input.shape[0]
    for b in range(input.shape[0]):
        for i in range(gradients.shape[0]):
            for j in range(gradients.shape[1]):
                gradients[i, j] += (-1 * (target[b, j] - pred[b, j]) * 2 * input[b, i] / gradients.shape[1] ) / batch_size

    return gradients

In [ ]:
fc = fc_layer(28**2, 10)

lr=1e-2
batch_size = 4
num_samples = len(train_data)
losses = []

for i in range(num_samples // batch_size):
    input = np.zeros((batch_size, 28**2))
    targets = np.zeros((batch_size, 10))

    for j in range(batch_size):
        # load img
        img = np.array(train_data.__getitem__(i*batch_size + j)[0]) / 255.
        img = img.flatten()
        input[j] = img

        # load labels
        label = np.array(train_data.__getitem__(i*batch_size + j)[1])
        label = label_one_hot_encoding(label)
        targets[j] = label

    pred = fc.forward(input)
    pred = softmax(pred)
    loss = mse(targets, pred)
    losses.append(loss)

    gradients = backward_mse_to_fc(input, targets, pred, fc.weights)
    fc.weights -= gradients*lr

In [26]:
test_data = MNIST(root="./", train=False, transform=None, target_transform=None, download=True)


10000


In [29]:
num_test_samples = len(test_data)
results = []

for i in range(10000):
    img = np.array(test_data.__getitem__(i)[0]) / 255.
    img = img.flatten()
    img = np.expand_dims(img, 0)

    label = np.array(test_data.__getitem__(i)[1])
    label = label_one_hot_encoding(label)
    label = np.expand_dims(label, 0)

    pred = fc.forward(img)
    pred = softmax(pred)

    if pred.argmax() == label.argmax():
        results.append(1)
    else:
        results.append(0)

print(np.mean(results))

0.8358
